# Weather Data Script
This is the script used to collect weather data (from the start of 2019 to 10/31/19) from [Weather Underground](https://www.wunderground.com/) to be used in the project "To Everything There is a Season : Using Weather Data and Demographic Information in the Predictive Modeling of Crimes in Dallas, Texas" by Ashley Steele. 

The basis for this script was borrowed from [this](https://stackoverflow.com/questions/39892710/using-pandas-to-scrape-weather-data-from-wundergound) post and modified to fit the needs of this project.

In [1]:
# Library import 
import io
import requests
import pandas as pd
from dateutil import parser, rrule
from datetime import datetime, time, date
import time

In [3]:
# Iterating through this script for each year of data we need & creating a new file by year
def getRainfallData(station, day, month, year):
    """
    Function to return a data frame of minute-level weather data for a single Wunderground PWS station.

    Args:
        station (string): Station code from the Wunderground website
        day (int): Day of month for which data is requested
        month (int): Month for which data is requested
        year (int): Year for which data is requested

    Returns:
        Pandas Dataframe with weather data for specified station and date.
    """
    url = "http://www.wunderground.com/weatherstation/WXDailyHistory.asp?ID={station}&day={day}&month={month}&year={year}&graphspan=day&format=1"
    full_url = url.format(station=station, day=day, month=month, year=year)

    # Request data from wunderground data
    response = requests.get(full_url)
    data = response.text
    # remove the excess <br> from the text data
    data = data.replace('<br>', '')

    # Convert to pandas dataframe (fails if issues with weather station)
    try:
        dataframe = pd.read_csv(io.StringIO(data), index_col=False)
        dataframe['station'] = station
    except Exception as e:
        print("Issue with date: {}-{}-{} for station {}".format(day,month,year, station))
        return None

    return dataframe

# Generate a list of all of the dates we want data for
start_date = "2019-01-01"
end_date = "2019-10-31"
start = parser.parse(start_date)
end = parser.parse(end_date)
dates = list(rrule.rrule(rrule.DAILY, dtstart=start, until=end))

# Create a list of stations here to download data for
stations = ["KTXIRVIN10"]
# Set a backoff time in seconds if a request fails
backoff_time = 10
data = {}

# Gather data for each station in turn and save to CSV.
for station in stations:
    print("Working on {}".format(station))
    data[station] = []
    for date in dates:
        # Print period status update messages
        if date.day % 10 == 0:
            print("Working on date: {} for station {}".format(date, station))
        done = False
        while done == False:
            try:
                weather_data = getRainfallData(station, date.day, date.month, date.year)
                done = True
            except ConnectionError as e:
                # May get rate limited by Wunderground.com, backoff if so.
                print("Got connection error on {}".format(date))
                print("Will retry in {} seconds".format(backoff_time))
                time.sleep(10)
        # Add each processed date to the overall data
        data[station].append(weather_data)
    # Finally combine all of the individual days and output to CSV for analysis
    pd.concat(data[station]).to_csv(r'2019_weather.csv'.format(station))

Working on KTXIRVIN10
Working on date: 2019-01-10 00:00:00 for station KTXIRVIN10
Working on date: 2019-01-20 00:00:00 for station KTXIRVIN10
Working on date: 2019-01-30 00:00:00 for station KTXIRVIN10
Working on date: 2019-02-10 00:00:00 for station KTXIRVIN10
Working on date: 2019-02-20 00:00:00 for station KTXIRVIN10
Working on date: 2019-03-10 00:00:00 for station KTXIRVIN10
Working on date: 2019-03-20 00:00:00 for station KTXIRVIN10
Working on date: 2019-03-30 00:00:00 for station KTXIRVIN10
Working on date: 2019-04-10 00:00:00 for station KTXIRVIN10
Working on date: 2019-04-20 00:00:00 for station KTXIRVIN10
Working on date: 2019-04-30 00:00:00 for station KTXIRVIN10
Working on date: 2019-05-10 00:00:00 for station KTXIRVIN10
Working on date: 2019-05-20 00:00:00 for station KTXIRVIN10
Working on date: 2019-05-30 00:00:00 for station KTXIRVIN10
Working on date: 2019-06-10 00:00:00 for station KTXIRVIN10
Working on date: 2019-06-20 00:00:00 for station KTXIRVIN10
Working on date: 2

C:\Users\gothv\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

